In [1]:
import get_data as driftlon_dataset
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, TimeDistributed
from datetime import datetime
import numpy as np

In [2]:
batch_size = 2000
match_depth = 50

(train_games, train_targets) = driftlon_dataset.get_data_batch(batch_size, matches_count=match_depth, normalized=True)

batch_size = len(train_games)

train_games = np.array(train_games)
train_targets = np.array(train_targets)

encoded = []
for target in train_targets:
    target_vector = [0, 0]
    target_vector[target] = 1
    encoded.append(target_vector)
train_targets = np.array(encoded)

#train_targets = train_targets.reshape(batch_size,1)

In [3]:
#configs
dropouts = [0.1*i for i in range(10)]
learning_rates = [0.00001, 0.00005, 0.0001, 0.0005, 0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]
init_accs = [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 1, 5]
configs = [(x, y, z) for x in dropouts for y in learning_rates for z in init_accs]
len(configs)

960

In [ ]:
activation = "tanh"
rnn_count = 128
cell_type = "LSTM"


for (dropout, learning_rate, init_acc) in configs:
    model = Sequential()

    model.add(LSTM(rnn_count, activation=activation, input_shape=(match_depth, 35), return_sequences=True))
    model.add(Dropout(dropout))

    model.add(LSTM(rnn_count))
    model.add(Dropout(dropout))

    model.add(Dense(rnn_count, activation=activation))
    model.add(Dense(rnn_count/4, activation=activation))
    model.add(Dense(2))

    #model.summary()

    loss = tf.keras.losses.BinaryCrossentropy()
    optimizer=tf.keras.optimizers.Adagrad(learning_rate=learning_rate, initial_accumulator_value=init_acc)
    metrics=['accuracy']

    model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

    suffix = 'dropout:' + str(dropout) + '-lr:' + str(learning_rate) + '-init_acc:' + str(init_acc)
    logdir='tb_logs/config_tries/' + suffix
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

    hist = model.fit(train_games, train_targets, batch_size=100, epochs=20, callbacks=[tensorboard_callback])

    if hist.history.get('accuracy')[-1] > 0.9:
        model.save('./neural_networks/configs-tries/' + suffix)


Epoch 1/20
20/20 [==============================] - 4s 202ms/step - loss: 3.9733 - accuracy: 0.7865
Epoch 2/20
20/20 [==============================] - 4s 190ms/step - loss: 3.5089 - accuracy: 0.8250
Epoch 3/20
20/20 [==============================] - 4s 189ms/step - loss: 3.3910 - accuracy: 0.8290
Epoch 4/20
20/20 [==============================] - 4s 189ms/step - loss: 3.3161 - accuracy: 0.8305
Epoch 5/20
20/20 [==============================] - 4s 192ms/step - loss: 3.2431 - accuracy: 0.8310
Epoch 6/20
20/20 [==============================] - 4s 203ms/step - loss: 3.1961 - accuracy: 0.8335
Epoch 7/20
20/20 [==============================] - 5s 258ms/step - loss: 3.1602 - accuracy: 0.8330
Epoch 8/20
20/20 [==============================] - 5s 249ms/step - loss: 3.1273 - accuracy: 0.8345
Epoch 9/20
20/20 [==============================] - 5s 248ms/step - loss: 3.0978 - accuracy: 0.8350
Epoch 10/20
20/20 [==============================] - 5s 262ms/step - loss: 3.0729 - accuracy: 0.8355

20/20 [==============================] - 4s 188ms/step - loss: 3.0666 - accuracy: 0.8110
Epoch 19/20
20/20 [==============================] - 4s 190ms/step - loss: 3.0501 - accuracy: 0.8130
Epoch 20/20
20/20 [==============================] - 4s 189ms/step - loss: 3.0337 - accuracy: 0.8130
Epoch 1/20
20/20 [==============================] - 4s 199ms/step - loss: 4.2190 - accuracy: 0.7075
Epoch 2/20
20/20 [==============================] - 4s 188ms/step - loss: 3.7521 - accuracy: 0.7795
Epoch 3/20
20/20 [==============================] - 4s 188ms/step - loss: 3.6546 - accuracy: 0.7845
Epoch 4/20
20/20 [==============================] - 4s 188ms/step - loss: 3.5755 - accuracy: 0.7895
Epoch 5/20
20/20 [==============================] - 4s 188ms/step - loss: 3.5007 - accuracy: 0.7940
Epoch 6/20
20/20 [==============================] - 4s 188ms/step - loss: 3.4382 - accuracy: 0.7955
Epoch 7/20
20/20 [==============================] - 4s 189ms/step - loss: 3.3710 - accuracy: 0.8025
Epoch 8/2

20/20 [==============================] - 4s 186ms/step - loss: 3.1030 - accuracy: 0.7880
Epoch 20/20
20/20 [==============================] - 4s 186ms/step - loss: 3.0874 - accuracy: 0.7880
Epoch 1/20
20/20 [==============================] - 4s 195ms/step - loss: 3.2533 - accuracy: 0.8195
Epoch 2/20
20/20 [==============================] - 4s 187ms/step - loss: 2.8547 - accuracy: 0.8465
Epoch 3/20
20/20 [==============================] - 4s 186ms/step - loss: 2.7347 - accuracy: 0.8535
Epoch 4/20
20/20 [==============================] - 4s 186ms/step - loss: 2.6736 - accuracy: 0.8550
Epoch 5/20
20/20 [==============================] - 4s 187ms/step - loss: 2.6236 - accuracy: 0.8550
Epoch 6/20
20/20 [==============================] - 4s 186ms/step - loss: 2.5849 - accuracy: 0.8585
Epoch 7/20
20/20 [==============================] - 4s 187ms/step - loss: 2.5528 - accuracy: 0.8595
Epoch 8/20
20/20 [==============================] - 4s 186ms/step - loss: 2.5131 - accuracy: 0.8600
Epoch 9/20

20/20 [==============================] - 4s 185ms/step - loss: 2.2341 - accuracy: 0.8450
Epoch 1/20
20/20 [==============================] - 4s 195ms/step - loss: 3.5543 - accuracy: 0.7690
Epoch 2/20
20/20 [==============================] - 4s 187ms/step - loss: 3.1598 - accuracy: 0.7830
Epoch 3/20
20/20 [==============================] - 4s 187ms/step - loss: 3.0233 - accuracy: 0.7895
Epoch 4/20
20/20 [==============================] - 4s 186ms/step - loss: 2.9204 - accuracy: 0.8020
Epoch 5/20
20/20 [==============================] - 4s 185ms/step - loss: 2.8146 - accuracy: 0.8060
Epoch 6/20
20/20 [==============================] - 4s 187ms/step - loss: 2.7434 - accuracy: 0.8110
Epoch 7/20
20/20 [==============================] - 4s 186ms/step - loss: 2.6918 - accuracy: 0.8115
Epoch 8/20
20/20 [==============================] - 4s 186ms/step - loss: 2.6447 - accuracy: 0.8115
Epoch 9/20
20/20 [==============================] - 4s 186ms/step - loss: 2.6098 - accuracy: 0.8145
Epoch 10/20

20/20 [==============================] - 4s 195ms/step - loss: 2.9148 - accuracy: 0.8285
Epoch 2/20
20/20 [==============================] - 4s 188ms/step - loss: 2.4597 - accuracy: 0.8515
Epoch 3/20
20/20 [==============================] - 4s 187ms/step - loss: 2.3311 - accuracy: 0.8600
Epoch 4/20
20/20 [==============================] - 4s 186ms/step - loss: 2.2420 - accuracy: 0.8645
Epoch 5/20
20/20 [==============================] - 4s 187ms/step - loss: 2.1717 - accuracy: 0.8665
Epoch 6/20
20/20 [==============================] - 4s 187ms/step - loss: 2.1176 - accuracy: 0.8675
Epoch 7/20
20/20 [==============================] - 4s 188ms/step - loss: 2.0719 - accuracy: 0.8695
Epoch 8/20
20/20 [==============================] - 4s 187ms/step - loss: 2.0283 - accuracy: 0.8710
Epoch 9/20
20/20 [==============================] - 4s 186ms/step - loss: 1.9922 - accuracy: 0.8715
Epoch 10/20
20/20 [==============================] - 4s 186ms/step - loss: 1.9539 - accuracy: 0.8715
Epoch 11/2

...

20/20 [==============================] - 4s 192ms/step - loss: 2.0785 - accuracy: 0.8445
Epoch 11/20
20/20 [==============================] - 4s 193ms/step - loss: 2.0368 - accuracy: 0.8410
Epoch 12/20
20/20 [==============================] - 4s 194ms/step - loss: 2.0060 - accuracy: 0.8510
Epoch 13/20
20/20 [==============================] - 4s 192ms/step - loss: 1.9986 - accuracy: 0.8490
Epoch 14/20
20/20 [==============================] - 4s 191ms/step - loss: 1.9835 - accuracy: 0.8440
Epoch 15/20
20/20 [==============================] - 4s 194ms/step - loss: 1.9627 - accuracy: 0.8480
Epoch 16/20
20/20 [==============================] - 4s 206ms/step - loss: 1.9086 - accuracy: 0.8520
Epoch 17/20
20/20 [==============================] - 4s 210ms/step - loss: 1.8832 - accuracy: 0.8505
Epoch 18/20
20/20 [==============================] - 4s 209ms/step - loss: 1.9108 - accuracy: 0.8505
Epoch 19/20
20/20 [==============================] - 4s 208ms/step - loss: 1.9062 - accuracy: 0.8515
Ep

20/20 [==============================] - 4s 190ms/step - loss: 1.0438 - accuracy: 0.8835
Epoch 12/20
20/20 [==============================] - 4s 192ms/step - loss: 0.9793 - accuracy: 0.8875
Epoch 13/20
20/20 [==============================] - 4s 190ms/step - loss: 0.9228 - accuracy: 0.8875
Epoch 14/20
20/20 [==============================] - 4s 194ms/step - loss: 0.8780 - accuracy: 0.8860
Epoch 15/20
20/20 [==============================] - 4s 194ms/step - loss: 0.8422 - accuracy: 0.8865
Epoch 16/20
20/20 [==============================] - 4s 193ms/step - loss: 0.7814 - accuracy: 0.8880
Epoch 17/20
20/20 [==============================] - 4s 194ms/step - loss: 0.7417 - accuracy: 0.8870
Epoch 18/20
20/20 [==============================] - 4s 190ms/step - loss: 0.7006 - accuracy: 0.8870
Epoch 19/20
20/20 [==============================] - 4s 190ms/step - loss: 0.6685 - accuracy: 0.8865
Epoch 20/20
20/20 [==============================] - 4s 193ms/step - loss: 0.6253 - accuracy: 0.8870
Ep

20/20 [==============================] - 4s 189ms/step - loss: 1.3931 - accuracy: 0.8700
Epoch 13/20
20/20 [==============================] - 4s 189ms/step - loss: 1.3676 - accuracy: 0.8685
Epoch 14/20
20/20 [==============================] - 4s 189ms/step - loss: 1.3438 - accuracy: 0.8740
Epoch 15/20
20/20 [==============================] - 4s 190ms/step - loss: 1.3077 - accuracy: 0.8775
Epoch 16/20
20/20 [==============================] - 4s 189ms/step - loss: 1.2789 - accuracy: 0.8765
Epoch 17/20
20/20 [==============================] - 4s 189ms/step - loss: 1.2550 - accuracy: 0.8700
Epoch 18/20
20/20 [==============================] - 4s 190ms/step - loss: 1.2348 - accuracy: 0.8765
Epoch 19/20
20/20 [==============================] - 4s 193ms/step - loss: 1.2017 - accuracy: 0.8760
Epoch 20/20
20/20 [==============================] - 4s 190ms/step - loss: 1.1924 - accuracy: 0.8745
Epoch 1/20
20/20 [==============================] - 4s 215ms/step - loss: 2.2000 - accuracy: 0.8520
Epo

20/20 [==============================] - 4s 188ms/step - loss: 0.5400 - accuracy: 0.8865
Epoch 14/20
20/20 [==============================] - 4s 188ms/step - loss: 0.5191 - accuracy: 0.8870
Epoch 15/20
20/20 [==============================] - 4s 189ms/step - loss: 0.5105 - accuracy: 0.8865
Epoch 16/20
20/20 [==============================] - 4s 188ms/step - loss: 0.5057 - accuracy: 0.8865
Epoch 17/20
20/20 [==============================] - 4s 188ms/step - loss: 0.4727 - accuracy: 0.8870
Epoch 18/20
20/20 [==============================] - 4s 189ms/step - loss: 0.4675 - accuracy: 0.8870
Epoch 19/20
20/20 [==============================] - 4s 188ms/step - loss: 0.4888 - accuracy: 0.8845
Epoch 20/20
20/20 [==============================] - 4s 188ms/step - loss: 0.4674 - accuracy: 0.8865
Epoch 1/20
20/20 [==============================] - 4s 207ms/step - loss: 1.9238 - accuracy: 0.8470
Epoch 2/20
20/20 [==============================] - 4s 206ms/step - loss: 1.4591 - accuracy: 0.8715
Epoc

20/20 [==============================] - 4s 188ms/step - loss: 1.0641 - accuracy: 0.8820
Epoch 15/20
20/20 [==============================] - 4s 189ms/step - loss: 1.0381 - accuracy: 0.8820
Epoch 16/20
20/20 [==============================] - 4s 190ms/step - loss: 1.0259 - accuracy: 0.8835
Epoch 17/20
20/20 [==============================] - 4s 188ms/step - loss: 0.9580 - accuracy: 0.8815
Epoch 18/20
12/20 [=================>............] - ETA: 1s - loss: 0.9462 - accuracy: 0.8767

In [37]:
    model.save('./neural_networks/saved_models/model-temp' )

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ./neural_networks/saved_models/model-gru0,8/assets


In [58]:
test_batch_size = 200

(test_games, test_targets) = driftlon_dataset.get_data_batch(test_batch_size)
test_games = np.array(test_games)
test_targets = np.array(test_targets)

test_encoded = []
for target in test_targets:
    target_vector = [0, 0]
    target_vector[target] = 1
    test_encoded.append(target_vector)
test_targets = np.array(test_encoded)

#train_targets = train_targets.reshape(batch_size,1)

In [60]:
model.evaluate(test_games, test_targets)

7/7 [==============================] - 0s 33ms/step - loss: 0.6846 - accuracy: 0.5900


[0.6845784187316895, 0.5899999737739563]

In [59]:
x = model.predict(test_games)
x

array([[ 0.39872792,  0.5368268 ],
       [ 0.39959332,  0.5380729 ],
       [ 0.39999634,  0.5386521 ],
       [ 0.39442286,  0.5306202 ],
       [ 0.3995553 ,  0.53801805],
       [ 0.3994433 ,  0.537857  ],
       [ 0.39935836,  0.5377346 ],
       [ 0.39949986,  0.5379383 ],
       [ 0.39917573,  0.5374717 ],
       [ 0.39944777,  0.53786343],
       [ 0.39929298,  0.5376392 ],
       [ 0.41271073,  0.55687   ],
       [ 0.39872393,  0.53682095],
       [ 0.3988333 ,  0.53697866],
       [ 0.39942375,  0.5378287 ],
       [ 0.39888895,  0.53705865],
       [ 0.39953014,  0.53798205],
       [ 0.39936855,  0.5377493 ],
       [ 0.39174986,  0.52675366],
       [ 0.40717873,  0.54884297],
       [ 0.39952534,  0.53797513],
       [ 0.39941156,  0.5378112 ],
       [ 0.39946154,  0.5378831 ],
       [ 0.40016523,  0.5388897 ],
       [ 0.39911023,  0.53737736],
       [ 0.4017165 ,  0.5411083 ],
       [ 0.3995059 ,  0.5379471 ],
       [ 0.39961284,  0.53810114],
       [ 0.39991432,

In [61]:
res = []
for elem in x:
    elem = list(elem)
    res.append(elem.index(max(elem)))
res.count(0)


1